In [15]:
import glob
import json
import random
import numpy as np
import pandas as pd
import re
import nltk

item = glob.glob(r'CORD-19-research-challenge/**/pdf_json/*.json',recursive=True)

random_d = np.random.choice(item, int(len(item)*.1))

In [39]:
import os
           
jsons_data = pd.DataFrame(columns=['title'])

for index, js in enumerate(random_d):
    
    with open(js) as json_file:
        
        json_text = json.load(json_file)
        
        for a in json_text['body_text']:
            
            only_text = a['text']

            #title = json_text['metadata']['title']
   
            jsons_data.loc[index]=[only_text]

print(jsons_data)
len(jsons_data)
jsons_data.describe()

                                                  title
0     The recent addition of chocolate-fl avoured in...
1     Numerous coronaviruses have been identified, a...
2     LCMV can be cleared from the brain parenchyma ...
3                         Insights & Perspectives .....
4     Present study shows that infections caused by ...
...                                                 ...
4589  In summary, we propose an OMZ-triggered signal...
4590  In conclusion, our study had manifested severa...
4591  Supplemental material for this article may be ...
4592  Lung representative sections were fixed with 4...
4593  Our results indicate that properly designed DN...

[4594 rows x 1 columns]


,title
count,4594
unique,4273
top,None.
freq,22


In [40]:
jsons_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4594 entries, 0 to 4593
Data columns (total 1 columns):
title    4594 non-null object
dtypes: object(1)
memory usage: 231.8+ KB


In [43]:
jsons_data

,title
0,The recent addition of chocolate-fl avoured in...
1,"Numerous coronaviruses have been identified, a..."
2,LCMV can be cleared from the brain parenchyma ...
3,Insights & Perspectives .....
4,Present study shows that infections caused by ...
...,...
4589,"In summary, we propose an OMZ-triggered signal..."
4590,"In conclusion, our study had manifested severa..."
4591,Supplemental material for this article may be ...
4592,Lung representative sections were fixed with 4...


In [18]:
result = []
for i in jsons_data['title']:
    result.append(i)

In [45]:
#result

In [19]:
import numpy as np

stop_words = nltk.corpus.stopwords.words('english')
stop_words
stop_words[:20]
len(stop_words)

179

In [20]:
def normalize_document(txt):
    txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt, re.I|re.A)
    txt = txt.lower()
    txt = txt.strip()
    tokens = nltk.word_tokenize(txt)
    clean_tokens = [t for t in tokens if t not in stop_words]
    return ' '.join(clean_tokens)

normalize_corpus = np.vectorize(normalize_document)
# New function that can take a list of items
norm_corpus = normalize_corpus(list(result))

In [21]:
len(norm_corpus)

4594

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tf.fit_transform(norm_corpus)

tfidf_matrix.shape

(4594, 271272)

Silhoutte test #11 was the optimal number of cluster for this model.

In [25]:

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

num_clus = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]

for num in num_clus:
    
    newK = KMeans(n_clusters = num)
    
    sil_Y = newK.fit_predict(tfidf_matrix)

    my_score = silhouette_score(tfidf_matrix, sil_Y)
    
    #print("For no of clusters =", num, 
          #" The average silhouette_score is :", my_score)



For no of clusters = 2  The average silhouette_score is : 0.008328786597459741
For no of clusters = 3  The average silhouette_score is : 0.013592670304324555
For no of clusters = 4  The average silhouette_score is : 0.012618670700263396
For no of clusters = 5  The average silhouette_score is : 0.017800349929007295
For no of clusters = 6  The average silhouette_score is : 0.01807042107971744
For no of clusters = 7  The average silhouette_score is : 0.013390923272816192
For no of clusters = 8  The average silhouette_score is : 0.019194296521309457
For no of clusters = 9  The average silhouette_score is : 0.02107087529878328
For no of clusters = 10  The average silhouette_score is : 0.02095134726019872
For no of clusters = 11  The average silhouette_score is : 0.0227980754768986
For no of clusters = 12  The average silhouette_score is : 0.019565097073561636
For no of clusters = 13  The average silhouette_score is : 0.01590311109117898
For no of clusters = 14  The average silhouette_score 

In [57]:

'''

For no of clusters = 2  The average silhouette_score is : 0.008328786597459741
For no of clusters = 3  The average silhouette_score is : 0.013592670304324555
For no of clusters = 4  The average silhouette_score is : 0.012618670700263396
For no of clusters = 5  The average silhouette_score is : 0.017800349929007295
For no of clusters = 6  The average silhouette_score is : 0.01807042107971744
For no of clusters = 7  The average silhouette_score is : 0.013390923272816192
For no of clusters = 8  The average silhouette_score is : 0.019194296521309457
For no of clusters = 9  The average silhouette_score is : 0.02107087529878328
For no of clusters = 10  The average silhouette_score is : 0.02095134726019872
For no of clusters = 11  The average silhouette_score is : 0.0227980754768986 <----------------The optimal number of clusters
For no of clusters = 12  The average silhouette_score is : 0.019565097073561636
For no of clusters = 13  The average silhouette_score is : 0.01590311109117898
For no of clusters = 14  The average silhouette_score is : 0.021824055459213842
For no of clusters = 15  The average silhouette_score is : 0.020243127477760803

''''''

In [26]:
from sklearn.cluster import KMeans
#help(sklearn.cluster)
#help(KMeans)

km = KMeans(n_clusters=11, max_iter=5, n_init=5, random_state=10)
km.fit(tfidf_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=5,
       n_clusters=11, n_init=5, n_jobs=None, precompute_distances='auto',
       random_state=10, tol=0.0001, verbose=0)

In [27]:
%time
%timeit
km.labels_

Wall time: 0 ns


array([0, 0, 9, ..., 2, 0, 0])

In [28]:
jsons_data['kmeans_cluster'] = km.labels_

In [32]:
movie_clusters = (jsons_data[['title', 'kmeans_cluster']]
                  .sort_values(by=['kmeans_cluster', 'title'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(20))

In [33]:
feature_names = tf.get_feature_names()

In [34]:
topn_features = 9
ordered_centroids = km.cluster_centers_.argsort()[:, ::-1]
ordered_centroids

array([[ 44959, 150071, 235002, ..., 182060,  85579, 271271],
       [268744, 111916, 238232, ..., 178697, 178696,      0],
       [148597, 171052, 237957, ..., 179443, 179442,      0],
       ...,
       [ 56738,  56734,  67885, ..., 180835, 180834,      0],
       [165931,  90011,  19266, ..., 176845, 176844,      0],
       [189407, 189466,  37158, ..., 180712, 180711,      0]], dtype=int64)

In [37]:
num_clust_choice = 11
for cluster_num in range(num_clust_choice):
    key_terms = [feature_names[index] 
                        for index in ordered_centroids[cluster_num, :topn_features]]
    phrases = movie_clusters[movie_clusters['kmeans_cluster'] == cluster_num].values.tolist()
    print('CLUSTER #'+str(cluster_num+1))
    print('Key Terms:', key_terms)
    print('Key Phrases/Sentences:', phrases)
    print('-'*80)

CLUSTER #1
Key Terms: ['cells', 'may', 'study', 'protein', 'infection', 'virus', 'viral', 'also', 'using']
Key Phrases/Sentences: [['되었고, 본격적으로 시행된 것은 1957 년 2 월 28 일 전염병예방법 시행령이 제정된 시점이라고 볼 수 있다 [8] . 역학조사관 Central  Internal medicine  2  2  2  3  1  3  1  2 16   Pediatrics  1  1  1  1  1  2  2  1  3  2  1  1  17   Family medicine  1  2  1  2  1  1  8   Preventive medicine  1  3  4   Neurology  1  1  2  1  2  1  2 10 Occupational and Environmental medicine **  3  2  1  1  2  1  1  2  1  3  4  1  2 24   General practitioner  1  1  1  3  2  4  1  1  1 15   No information  7  7  4  1  19   Subtotal  14 15  7  18 11 11 15 13 11 10 10 11  6  8  11  9  9 189   Total  18 19 10 23 13 14 19 15 16 14 11 19  9  13 17 12 13 ', 0], ['■ Identifying more effective ways to communicate knowledge of infection control to hospital personnel, especially with regard to handwashing, aseptic use of devices, and antibiotic therapy, and to apply it consistently in all hospitals would have vast immediate benefi 